In [82]:
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import prep_for_model_runs as prep
import model_params_class as mp
import run_models as run


In [83]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Deterministic SIR model for multi-groups 
See: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4002176/


The model works like this: 
- Let's say we have N groups with size n_0 .... n_N
- Define four vectors (S = 'Susceptible, I = Infected, R = Recovered, D = Died) for each of the groups
    - for example: $\vec{I} = < I_0, I_1 \dots I_N >$ is the vector for the number of currently infected people in each subpopulation
- Assume a "contact matrix" $C = c_{ij} $ where $c_{ij}$ is the expected rate of contacts between groups i and j.
    - for example if $c_{i,j}$ = 0.1 that means that average number of contacts that an individual in group i has with group j is 0.1 --> i.e. there is a 1/10 chance they see anyone 
- Assume a "transmission matrix" $T = t_{ij}$ where $t_{ij}$  is the expected transmission between groups i and j if they make contact
    - The following code assumes that this is 1 (e.g. transmission rate is rolled into contact matrix) and is constant across groups. 
    - In a more sophisticated model, this assumption is not true. The transmission rate should not even be reciprocal. For example, if police are wearing masks when they interact with a community member, but there hands are equipment are dirty transmission rate could be higher in one direction. Transmission rate could also be higher for different types of interactions
- $\vec{\gamma}$ is the recovery rate .. e.g. $\gamma_i$ the likelihood that on a given day a sick person in group i recovers. We are using 1/14
- $\vec{\mu}$ is the death rate (divided by average disease duration) for each group

Combining contact and transmission, we can define a vector $\vec{\lambda}$ where $\lambda_i$ is the new infection rate (called the force of infection in the literature). 

- Force of infection = contact x transmission or $\lambda_i = \sum_j^N c_{ij} * t_{ij} $

Given $\vec{\lambda}$ the vector with contact rate * transmission rate, we can define our differential equations as follows:(Note that $t$ is current time and $t-1$ is previous time point. 

- Sub population totals : $\vec{N} = \vec{I_t} + \vec{R_t} + \vec{S_t} + \vec{D_t}$
- Susceptible number of people by sup-population: $\vec{S_t} = \vec{S_{t-1}} - \vec{I_{t-1}}$
- Infected by subpopulation $\vec{I_t} = \vec{I_{t-1}}+  \vec{\lambda} * \vec{S_{t-1}} * \frac{\vec{I_{t-1}}}{\vec{N}} - \vec{\gamma} \vec{I_{t-1}} - \vec{\mu} \vec{I_{t-1}}$ 
    - $\vec{I_t} =$ infected people before + newly infected people - infected people who recover
    - newly infected people = susceptible population * contact rate * transmission rate * proportion of population infected 
    - $\frac{\vec{I_{t-1}}}{\vec{N}}$ is the proportion of members of each group that are infected
- Recovered by Group: $\vec{R_t} = \vec{R_{t-1}} + \vec{\gamma} \vec{I_{t-1}}$

The implementation of this model does not used died, which simplifies it. 

In [84]:
#TODO add markdown with summary statistics meaning

In [85]:
color_matrix = {'White': 'k',
                'White_Forced_Labour' : 'r',
                'Black' : 'g',
                'Black_Forced_Labour': 'b',
                'Black_Prison' : 'k',
                'White_Police' : 'o',
                'Black_Police' : 'p',
                'White_Prison': 'y',
                'Total_Residents': 'g'}

In [117]:
def clean_df_names(df):
    df.columns = df.columns.str.strip()
    df.index = df.index.map(lambda s: s.strip())
    return df

In [118]:
"""Run multiple versions of the model, varying certain parameters to quantify uncertainty

params
------
base_dir: String
    the name of the directory where contact matrices and group size matrices are stored
    
starting_params: object of class ModelParams
    contains the initial parameters for the original model run
    
prison_peak_date: Int
    the day on which the prison infection rate peaks
    
days: Int
    the number of days to run the model for
    
returns
-------
monster_summary_stats: a DataFrame of all of the summary statistics for the model runs
infection_maps: ???
pd.concat(pop_sizes): ???
"""
def run_with_uncertainty(base_dir, starting_params, prison_peak_date, days):
    monster_summary_stats = pd.DataFrame({})
    infection_maps = {}
    pop_sizes = []

    for prison_infection_rate in [25, .35, .45]:
        for pc in [5, 7.5, 10, 12.5,15]:
            for pgrp in [13.8, 14.9, 16, 17, 18.2]:
                
                params = starting_params.add_uncertainty_params(prison_infection_rate, pc, pgrp)
                
                # get contact matrix
                contact_data_post_sip = pd.read_csv(
                    os.path.join(base_dir, prep.get_CONTACT_MATRIX_POST_LOCKDOWN(pc, pgrp)), 
                    index_col=0)
                
                
                
  
                contact_data_pre_SIP = clean_df_names(pd.read_csv(os.path.join(base_dir, prep.get_CONTACT_MATRIX_PRE_LOCKDOWN(pc,pgrp)),
                                                   index_col=0))
    
      
                
                group_size_data = clean_df_names(pd.read_csv(
                    os.path.join(base_dir,prep.get_GROUP_SIZE_PATH(params.police_group_size))).set_index('Group'))
                eq_group_size_data = clean_df_names(pd.read_csv(
                    os.path.join(base_dir, prep.get_GROUP_SIZE_EQ_PATH(params.police_group_size))).set_index('Group'))
                            
                summary, infection_df, pop_size_df = run.run_models(
                    base_dir, params, days, group_size_data,
                    eq_group_size_data,contact_data_pre_SIP,
                    contact_data_post_sip, prison_peak_date)                                             

                infection_maps[params.get_name()] = infection_df
                pop_sizes.append(pop_size_df)
                
                summary['Initial_Infections'] = starting_params.initial_infection_multiplier
                summary['Lockdown_Date'] = starting_params.sip_start_date
                summary['Prison_Rate'] = starting_params.prison_infection_rate
                summary['Police_Contact_Rate'] = pc
                summary['Police_Group_Size'] = pgrp 
                monster_summary_stats= monster_summary_stats.append(summary)
    return monster_summary_stats, infection_maps, pd.concat(pop_sizes)
    

In [119]:
PRISON_PEAK_DATE = 3*7 #I think we'll want to change this to 35 once we're ready to publish the code;
#I think that's the version Tracey/Phil used when writing
TRANSMISSION_RATE = 0.015
SIP_START_DATE  = 14
INITIAL_INFECTION_RATE =10
BASE_DIR = '../input_data/pc_pgrp_matrices/'
starting_params = mp.ModelParams(TRANSMISSION_RATE, SIP_START_DATE, INITIAL_INFECTION_RATE)


In [120]:
CONTACT_MATRIX_PATH = 'Contact_Matrix_Post_SIP.csv'
PRE_SOCIAL_DISTANCE_CONTACT_MATRIX = "Contact_Matrix_Pre_SIP.csv"

DAYS = 120

In [124]:
monster_summary_stats, infection_maps, pop_sizes = run_with_uncertainty(
    BASE_DIR, starting_params, PRISON_PEAK_DATE, DAYS)


In [125]:
monster_summary_stats.head()

,model_tag,cumulative_infected_40_days,cumulative_rate_40_days,active_rate_before_peak,cumulative_before_peak,cumulative_rate_before_peak,days_peak,peak_active_infected_rate,cumulative_peak,cumulative_rate_peak,active_rate_after_peak,cumulative_after_peak,cumulative_rate_after_peak,model_name,Initial_Infections,Lockdown_Date,Prison_Rate,Police_Contact_Rate,Police_Group_Size
Group,,,,,,,,,,,,,,,,,,,
Black,I10_PI25_L14___pc5_pgrp13.8,117589.0,0.203170,0.275107,257528.0,0.444956,54,0.275865,267408.0,0.462027,0.270847,294766.0,0.509296,original,10,14,25.0,5.0,13.8
Black_Forced_Labour,I10_PI25_L14___pc5_pgrp13.8,38746.0,0.424715,0.558754,81455.0,0.892870,49,0.618036,73516.0,0.805847,0.415940,85437.0,0.936519,original,10,14,25.0,5.0,13.8
White,I10_PI25_L14___pc5_pgrp13.8,228277.0,0.085173,0.177505,659912.0,0.246223,62,0.211054,999039.0,0.372756,0.200827,816613.0,0.304690,original,10,14,25.0,5.0,13.8
White_Forced_Labour,I10_PI25_L14___pc5_pgrp13.8,125765.0,0.370052,0.585615,294927.0,0.867795,50,0.623668,271622.0,0.799223,0.453209,312417.0,0.919258,original,10,14,25.0,5.0,13.8
Total_Residents,I10_PI25_L14___pc5_pgrp13.8,510376.0,0.138313,0.239827,1293822.0,0.350629,55,0.242472,1404691.0,0.380675,0.240373,1509234.0,0.409007,original,10,14,25.0,5.0,13.8


In [126]:
pop_sizes.to_csv('group_sizes.csv')

In [127]:
monster_path = 'final_output/all_model_summary_stats.csv'
monster_summary_stats.to_csv(monster_path)

FileNotFoundError: [Errno 2] No such file or directory: 'final_output/all_model_summary_stats.csv'

In [128]:
monster_summary_stats.groupby('model_name').size()

model_name
eq_forced_labour       450
eq_police              450
eq_police_prison       450
eq_police_prison_fl    450
eq_prison              450
no_forced_labour       450
no_police              450
no_police_prison_fl    450
no_prison              450
no_prison_or_police    450
original               450
dtype: int64

In [129]:
# Graph one of the models so we can check on the group sizes 

In [130]:
infection_maps.keys()

dict_keys(['I10_PI25_L14___pc5_pgrp13.8', 'I10_PI25_L14___pc5_pgrp14.9', 'I10_PI25_L14___pc5_pgrp16', 'I10_PI25_L14___pc5_pgrp17', 'I10_PI25_L14___pc5_pgrp18.2', 'I10_PI25_L14___pc7.5_pgrp13.8', 'I10_PI25_L14___pc7.5_pgrp14.9', 'I10_PI25_L14___pc7.5_pgrp16', 'I10_PI25_L14___pc7.5_pgrp17', 'I10_PI25_L14___pc7.5_pgrp18.2', 'I10_PI25_L14___pc10_pgrp13.8', 'I10_PI25_L14___pc10_pgrp14.9', 'I10_PI25_L14___pc10_pgrp16', 'I10_PI25_L14___pc10_pgrp17', 'I10_PI25_L14___pc10_pgrp18.2', 'I10_PI25_L14___pc12.5_pgrp13.8', 'I10_PI25_L14___pc12.5_pgrp14.9', 'I10_PI25_L14___pc12.5_pgrp16', 'I10_PI25_L14___pc12.5_pgrp17', 'I10_PI25_L14___pc12.5_pgrp18.2', 'I10_PI25_L14___pc15_pgrp13.8', 'I10_PI25_L14___pc15_pgrp14.9', 'I10_PI25_L14___pc15_pgrp16', 'I10_PI25_L14___pc15_pgrp17', 'I10_PI25_L14___pc15_pgrp18.2', 'I10_PI0.35_L14___pc5_pgrp13.8', 'I10_PI0.35_L14___pc5_pgrp14.9', 'I10_PI0.35_L14___pc5_pgrp16', 'I10_PI0.35_L14___pc5_pgrp17', 'I10_PI0.35_L14___pc5_pgrp18.2', 'I10_PI0.35_L14___pc7.5_pgrp13.8', 'I1

In [131]:
i = infection_maps['I10_PI0.45_L28___pc15_pgrp18.2']
pop_for_model = pop_sizes[pop_sizes['name'] == 'I10_PI0.45_L28___pc15_pgrp18.2']
p = pop_for_model.loc['original']


KeyError: 'I10_PI0.45_L28___pc15_pgrp18.2'

In [ ]:
i.head()

In [ ]:
i[i['model_name'] == 'original']['White']/p['White']

In [ ]:
pop_sizes.head()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize = (20, 20))
fig.suptitle("Black Residents vs. White Residents")

model_colors = {
    'original' : 'k',
  #  'eq_forced_labour': 'b',
    'no_prison': 'b',
    'no_police': 'g',
    'no_prison_or_police': 'r',
    'no_forced_labour' : 'orange'
}

for k in model_colors.keys():
    df = i[i['model_name'] == k]
    pop_size = pop_for_model.loc[k]
    #print(k + "   " + str(pop_size['Black_Forced_Labour']))
    x = df['Black_Forced_Labour']/pop_size['Black_Forced_Labour']
    y = df['Black']/pop_size['Black']
    z = (df['Black'] + df['Black_Forced_Labour'])/ (pop_size['Black'] + pop_size['Black_Forced_Labour'])

    ax1.plot(df.index, x , model_colors[k], alpha=0.85, lw=3, label = 'BLACK_' + k)
    ax2.plot(df.index, y ,model_colors[k], alpha=0.85, lw=3, label='BLACK_' + k)
    ax3.plot(df.index, z, model_colors[k], alpha =0.85, lw=3, label =  "BLACK_" + k)

    if k in ['original', 'no_forced_labour']:
        z2 = (df['White'] + df['White_Forced_Labour'])/(pop_size['White'] + pop_size['White_Forced_Labour'])

        ax1.scatter(df.index, df['White_Forced_Labour']/pop_size['White_Forced_Labour'],
                    color = model_colors[k], lw = 2, label = 'WHITE '+k)
        ax2.scatter(df.index, df['White']/pop_size['White'], color = model_colors[k], lw = 2, label = 'WHITE '+ k)
        ax3.scatter(df.index, z2, color = model_colors[k], lw = 2, label = 'WHITE '+k)
    
ax2.set_xlabel('Time /days')

ax1.set_ylabel('Infection rate ')
ax2.set_ylabel('Infection rate ')
ax3.set_ylabel('Infection rate ')

ax1.set_title('Infection Rate Black Force Labour')
ax2.set_title('Infection Rate Black')
ax2.set_title('Infection Rate Black / White All population + Forced Labour')

ax1.legend(loc = 'best')
ax2.legend()
ax3.legend()
# ax2.set_ylim(0,0.75)
# ax1.set_ylim(0,0.75)

ax2.set_xlim(10,100)
ax1.set_xlim(10,100) 
ax2.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)
# ax.grid(b=True, which='major', c='w', lw=2, ls='-')
 
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize = (20, 25))
#fig.suptitle("Black Residents vs. White Residents")
plt.rc('font', size=16) 

model_colors = {
    'original' : 'k',
    'eq_forced_labour': 'b',
    'eq_police_prison': 'g',
    'eq_police_prison_fl': 'r',
    'no_forced_labour' : 'orange'
}
for k in model_colors.keys():
    df = i[i['model_name'] == k]
    pop_size = pop_for_model.loc[k]
    
    x = df['Black_Forced_Labour']
    y= df['Black']
    z = (df['Black'] + df['Black_Forced_Labour'])

    x2 = df['White_Forced_Labour']
    y2 = df['White'] 
    z2 = (df['White'] + df['White_Forced_Labour'])

    #/(pop_size['White'] + pop_size['White_Forced_Labour'])

    ax1.plot(df.index, x , model_colors[k], alpha=0.5, lw=3, label=k+'')
    ax2.plot(df.index, y ,model_colors[k], alpha=0.5, lw=3, label=k+'')
    ax3.plot(df.index, z, model_colors[k], alpha = 0.5, lw = 3, label = k)

#     if k in ['original', 'eq_police_prison_fl']:
#         ax1.scatter(df.index, x2 , color = model_colors[k], alpha=0.5, lw=2, label='WHITE ' + k)
#         ax2.scatter(df.index, y2 ,color = model_colors[k], alpha=0.5, lw=2, label='WHITE ' + k)
#         ax3.scatter(df.index, z2, color = model_colors[k], alpha = 0.5, lw = 2, label ='WHITE ' + k)

    
ax2.set_xlabel('Time /days')
ax2.set_ylabel('# Infections')
ax1.set_ylabel(' # Infections')
ax3.set_ylabel(' # Infections')

ax1.set_title('# Infections Black Forced Labour')
ax2.set_title('# Infections Black Non Forced Labour')
ax2.set_title('# Infections Infection Rate All Black / White (Population and non forced labour)')
# ax2.set_ylim(0,0.75)
# ax1.set_ylim(0,0.75)

ax2.set_xlim(10,100)
ax1.set_xlim(10,100) 
ax2.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)
# alegendrid(b=True, which='major', c='w', lw=2, ls='-')
 
legend = ax1.legend()
ax2.legend()
ax3.legend()
legend.get_frame().set_alpha(0.5)
plt.show()